<a href="https://colab.research.google.com/github/baiyinnamula/EasyToUnderstandPandas/blob/main/%E7%AC%AC6%E7%AB%A0_Pandas%E5%88%86%E7%BB%84%E8%81%9A%E5%90%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>